<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/facevidblur1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FaceVidBlur

### Notebook by [Eyal Gruss](https://eyalgruss.com), [@eyaler](twitter.com/eyaler)

Blur faces in video + optionally draw facial landmarks

More notebooks: [github.com/eyaler/avatars4all](https://github.com/eyaler/avatars4all)

Shortcut here: [tfi.la/blur](https://tfi.la/blur)

Something not working? Open an [issue](https://github.com/eyaler/avatars4all/issues)

If you find my work useful please consider supporting me via [GitHub Sponsors](https://github.com/sponsors/eyaler) or [PayPal](https://www.paypal.com/donate/?hosted_button_id=LNJ6F3FR79ARE)

In [ ]:
#@title Setup

import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!pip install git+https://github.com/ytdl-org/youtube-dl
!pip install git+https://github.com/1adrianb/face-alignment

In [ ]:
#@title Optionally mount Google Drive (MARK CHECKBOX) { run: "auto" }
mount_google_drive = False #@param {type:"boolean"}
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  print('path is /content/drive/MyDrive')

In [ ]:
#@title Blur faces in video

video_url = 'https://www.youtube.com/watch?v=SoAKSHcrDGg' #@param {type: 'string'}
#@markdown (leave empty to upload file - A BUTTON WILL APPEAR BELOW, or link to youtube / vimeo / video url / path to video on mounted drive [/content/drive/MyDrive/...] / path to video on colab)
i_just_uploaded_a_file_and_i_want_to_reuse_that_instead_of_uploading_a_new_one = False #@param {type: 'boolean'}
face_order = 'left-to-right then top-to-bottom' #@param ['left-to-right then top-to-bottom', 'top-to-bottom then left-to-right']
#@markdown (one of these orderings may be more stable for your video, as order is recalculated per frame)
face_num = '' #@param {type: 'string'}
#@markdown (leave empty to blur all faces, or use face_num = 1,2,... to blur one or more faces from all the faces ordered as above)
safe_mode = True #@param {type: 'boolean'}
#@markdown (hide neighbour faces when the number of faces changes)
min_conf = 0.9 #@param {type: 'number'}
#@markdown (lower min_conf if faces are not detected, raise min_conf if there are false detections)
radius_factor = 1 #@param {type: 'number'}
expose_eyes_mouth = False #@param {type: 'boolean'}
draw_landmarks = True #@param {type: 'boolean'}
#@markdown (uncheck this if you don't want the face contours)
landmarks_hex = '0000FF' #@param {type: 'string'}
start_seconds = 0 #@param {type: 'number'}
duration_seconds = 15 #@param {type: 'number'}
#@markdown (use duration_seconds = 0 for unrestricted duration)
max_width = 0 #@param {type: 'integer'}
#@markdown (use max_width = 0 for unrestricted width)
max_height = 0 #@param {type: 'integer'}
#@markdown (use max_height = 0 for unrestricted height)
output_filename = 'output.mp4' #@param {type: 'string'}

from time import time
start_time = time()

import os
from google.colab import files

need_dl = True
try:
  if orig_video and video_url == save_url and (video_url or i_just_uploaded_a_file_and_i_want_to_reuse_that_instead_of_uploading_a_new_one):
    need_dl = False
except:
  pass
if need_dl:
  if not video_url:
    %cd /content/sample_data
    try:
      uploaded = files.upload()
    except Exception:
      %cd /content
      raise
    for fn in uploaded:
      orig_video = os.path.abspath(fn)
      break
    else:
      need_dl = False
    %cd /content
  elif os.path.isfile(video_url):
    orig_video = os.path.abspath(video_url)
  elif os.path.isfile('/content/drive/MyDrive/' + video_url):
    orig_video = '/content/drive/MyDrive/' + video_url
  else:
    orig_video = '/content/orig_video.mp4'
    !rm -f $orig_video
    !youtube-dl --no-playlist -f "bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4[vcodec!*=av01]/bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo[ext=mp4]+bestaudio/mp4" "$video_url" --merge-output-format mp4 -o $orig_video
    if not os.path.exists(orig_video):
      orig_video = '/content/orig_video.' + video_url.rsplit('.', 1)[1]
      !wget "$video_url" -O $orig_video
    assert os.path.exists(orig_video)
  input_video = '/content/input_video' + os.path.splitext(orig_video)[-1]

need_fix_duration = True
try:
  if not need_dl and start_seconds == save_start_seconds and duration_seconds == save_duration_seconds:
    need_fix_duration = False
except:
  pass
if need_fix_duration:
  if start_seconds or duration_seconds:
    !ffmpeg -y -ss $start_seconds -t $duration_seconds -i "$orig_video" -f mp4 $input_video
  else:
    !cp "$orig_video" $input_video

save_url = video_url
save_start_seconds = start_seconds
save_duration_seconds = duration_seconds

import imageio.v3 as iio
from IPython.display import display, Image, Video, clear_output
import face_alignment
import numpy as np
from skimage.draw import ellipse, polygon, polygon_perimeter
import cv2

fa = face_alignment.FaceAlignment(landmarks_type=1)
fps = iio.immeta(input_video)['fps']

!rm -rf /content/in_frames
!mkdir -p /content/in_frames
!rm -f /content/numbers.png

face_parts = {'eyebrow1': slice(17, 22),
              'eyebrow2': slice(22, 27),
              'nose': slice(27, 31),
              'nostril': slice(31, 36),
              'eye1': slice(36, 42),
              'eye2': slice(42, 48),
              'lips': slice(48, 60),
              'teeth': slice(60, 68)
              }

step = len(landmarks_hex) // 3
landmarks_rgb = [int(landmarks_hex[i: i + step], 16) * 17**(step == 1) for i in range(0, len(landmarks_hex), step)]

face_num = [int(i or 0) - 1 for i in face_num.split(',')]
have_faces = False
max_face_conf = 0
max_face_conf_below = 0
min_face_conf_above = 1
for i, im in enumerate(iio.imiter(input_video)):
  landmarks, _, bboxes = fa.get_landmarks_from_image(im, return_bboxes=True)
  if bboxes is not None:
    lb = zip(landmarks, bboxes)
    max_face_conf = max(max_face_conf, *[b[-1] for b in bboxes])
    if min_conf:
      for b in bboxes:
        if b[-1] < min_conf:
          max_face_conf_below = max(max_face_conf_below, b[-1])
      lb = [(l, b) for l, b in lb if b[-1] >= min_conf]
    lb = sorted(lb, key=lambda x: (x[1][0], x[1][2], x[1][1], x[1][3]) if face_order == 'left-to-right then top-to-bottom' else (x[1][1], x[1][3], x[1][0], x[1][2]))
    if len(lb) > 1 and not have_faces:
      have_faces = True
      face_count = len(lb)
      numbers = im.copy()
      for j, (_, (x0, y0, x1, y1, _)) in enumerate(lb):
          cv2.putText(numbers, str(j + 1), (max(min(int(x1), im.shape[1]) - 20, 0), max(min(int(y1), im.shape[0]), 20)), 0, .7, (0, 255, 0), 2)
      iio.imwrite('/content/numbers.png', numbers, compress_level=1)
      clear_output()
      display(Image('/content/numbers.png'))
    if face_num != [-1]:
      diff = len(lb) - face_count
      if safe_mode and diff:
        safe_face_num = []
        for n in face_num:
          start = max(min(n, n + diff), max(safe_face_num, default=-1) + 1)
          end = max(max(n, n + diff), start) + 1
          safe_face_num += list(range(start, end))
      else:
        safe_face_num = face_num
      lb = [lb[n] for n in safe_face_num if len(lb) > n]
    orig = im.copy()
    for landmarks, (x0, y0, x1, y1, conf) in lb:
      min_face_conf_above = min(min_face_conf_above, conf)
      x = (x0+x1) / 2
      y = (y0+y1) / 2
      wr = (x1-x0) / 2 * radius_factor
      hr = (y1-y0) / 2 * radius_factor
      yy, xx = ellipse(y, x, hr, wr, im.shape)
      im[yy, xx] = np.median(orig[yy, xx], axis=0)
      if expose_eyes_mouth:
        for name, part in face_parts.items():
          if name in ('eye1', 'eye2', 'lips'):
            yy, xx = polygon(*list(zip(*landmarks[part]))[::-1], im.shape)
            im[yy, xx] = orig[yy, xx]
    if draw_landmarks:
      for landmarks, _ in lb:
        for part in face_parts.values():
          yy, xx = polygon_perimeter(*list(zip(*landmarks[part]))[::-1], im.shape)
          im[yy, xx] = landmarks_rgb
  iio.imwrite(f'/content/in_frames/frame_{i:06d}.png', im, compress_level=1)

!ffmpeg -y -framerate $fps -thread_queue_size 0 -i /content/in_frames/frame_%06d.png -i $input_video -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "scale=min(iw\,$max_width):min(ih\,$max_height):force_original_aspect_ratio=decrease:force_divisible_by=2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart "/content/$output_filename"

clear_output()
if have_faces:
  display(Image('/content/numbers.png'))
meta = iio.immeta('/content/' + output_filename)
print(f'took {(time()-start_time) / 60 :.1f} min. {orig_video=} output_video=/content/{output_filename} w={meta["size"][0]} h={meta["size"][1]} t={meta["duration"]} fps={meta["fps"]} {max_face_conf_below=:.2f} {min_face_conf_above=:.2f} {max_face_conf=:.2f}')
print('if video does not show below, you can still download it!')
display(Video('/content/' + output_filename, embed=True, html_attributes="autoplay controls loop"))

In [ ]:
#@title Download

from google.colab import files
files.download('/content/' + output_filename)